In [10]:
# Cell 1: Core Functions and Data Retrieval
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import time
import re
from datetime import datetime, timedelta
from scipy.signal import argrelextrema

# Set global variables to store data for access across cells
global nyse_tickers, daily_data, spy_data, breadth_data, ad_data, merged_data

def get_nyse_tickers(file_path='NYSE.csv'):
    """
    Retrieves NYSE tickers from a CSV file.
    
    Args:
        file_path (str): Path to the NYSE CSV file
    
    Returns:
        list: List of NYSE ticker symbols with proper formatting for yfinance
    """
    try:
        # Read the CSV file
        nyse_data = pd.read_csv(file_path)
        
        # Extract the Symbol column
        tickers = nyse_data['Symbol'].tolist()
        
        # Convert tickers to yfinance format
        tickers = convert_tickers_for_yfinance(tickers)
        
        print(f"Successfully retrieved {len(tickers)} NYSE ticker symbols")
        return tickers
    
    except Exception as e:
        print(f"Error retrieving NYSE tickers from CSV: {e}")
        return []

def convert_tickers_for_yfinance(tickers):
    """
    Filter out ticker symbols that contain hyphens or special characters
    to avoid double counting in A/D line analysis.
    
    Args:
        tickers (list): List of ticker symbols
        
    Returns:
        list: List of clean ticker symbols without special characters
    """
    clean_tickers = []
    excluded_count = 0
    
    for ticker in tickers:
        # Convert to string in case it's not already
        ticker_str = str(ticker)
        
        # Check if the ticker contains any special characters or hyphens
        if re.search(r'[\^/\.\-]', ticker_str):
            excluded_count += 1
            continue
        
        # Add only clean tickers to the result list
        clean_tickers.append(ticker_str)
    
    print(f"Kept {len(clean_tickers)} clean tickers, excluded {excluded_count} tickers with special characters")
    return clean_tickers

def get_daily_data(tickers, start_date, end_date, batch_size=500):
    """
    Retrieves daily price data for a list of tickers in batches to avoid API limitations.
    
    Args:
        tickers (list): List of ticker symbols
        start_date (str): Start date for data retrieval in YYYY-MM-DD format
        end_date (str): End date for data retrieval in YYYY-MM-DD format
        batch_size (int): Number of tickers to download at once
    
    Returns:
        pd.DataFrame: DataFrame with daily closing prices for each ticker
    """
    all_data = pd.DataFrame()
    
    # Process tickers in batches to avoid API limitations
    for i in range(0, len(tickers), batch_size):
        batch_tickers = tickers[i:i+batch_size]
        print(f"Downloading data for tickers {i+1} to {min(i+batch_size, len(tickers))}...")
        
        try:
            # Download daily data for the batch
            batch_data = yf.download(batch_tickers, start=start_date, end=end_date, progress=False)
            
            # If we have more than one ticker, we'll have a MultiIndex DataFrame
            if len(batch_tickers) > 1:
                batch_close = batch_data['Close']
            else:
                # For a single ticker, we need to handle differently
                batch_close = batch_data['Close'].to_frame(name=batch_tickers[0])
            
            # For the first batch, set this as our dataframe
            if all_data.empty:
                all_data = batch_close
            else:
                # For subsequent batches, join with existing data
                all_data = all_data.join(batch_close, how='outer')
            
            # Add a small delay to avoid hitting API limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Error downloading data for batch starting at index {i}: {e}")
    
    print(f"Successfully downloaded daily data with shape {all_data.shape}")
    return all_data

def get_spy_data(start_date, end_date):
    """
    Retrieves SPY ETF data for comparison with A/D line.
    Properly handles the multi-index structure returned by yfinance.
    
    Args:
        start_date (str): Start date for data retrieval in YYYY-MM-DD format
        end_date (str): End date for data retrieval in YYYY-MM-DD format
    
    Returns:
        pd.DataFrame: DataFrame with SPY daily data with proper column structure
    """
    try:
        # Download SPY data
        spy_data = yf.download('SPY', start=start_date, end=end_date, progress=False)
        
        # Fix multi-index columns if present
        if len(spy_data.columns.names) > 1:
            spy_data.columns = spy_data.columns.droplevel(1)
            
        print(f"Successfully downloaded SPY data with columns: {spy_data.columns.tolist()}")
        return spy_data
    except Exception as e:
        print(f"Error downloading SPY data: {e}")
        return pd.DataFrame()

def calculate_daily_advances_declines(daily_data):
    """
    Calculate the number of advancing and declining stocks each day.
    
    Args:
        daily_data (pd.DataFrame): DataFrame with daily closing prices for each ticker
    
    Returns:
        pd.DataFrame: DataFrame with daily counts of advances, declines, and unchanged
    """
    # Calculate daily price changes
    daily_changes = daily_data.pct_change()
    
    # Count advances, declines, and unchanged for each day
    advances = (daily_changes > 0).sum(axis=1)
    declines = (daily_changes < 0).sum(axis=1)
    unchanged = (daily_changes == 0).sum(axis=1)
    
    # Create DataFrame with results
    breadth_data = pd.DataFrame({
        'Advances': advances,
        'Declines': declines,
        'Unchanged': unchanged,
        'Total': advances + declines + unchanged,
        'Advance_Decline_Diff': advances - declines
    })
    
    print(f"Calculated daily breadth data with shape {breadth_data.shape}")
    return breadth_data

def calculate_ad_line(breadth_data, initial_value=0):
    """
    Calculate the Advance/Decline line and related technical indicators.
    
    Args:
        breadth_data (pd.DataFrame): DataFrame with daily advances and declines
        initial_value (int): Starting value for the A/D line
    
    Returns:
        pd.DataFrame: DataFrame with A/D line and technical indicators
    """
    # Create a copy of the input data
    ad_data = breadth_data.copy()
    
    # Calculate the A/D line (cumulative sum of daily A/D differences)
    ad_data['AD_Line'] = initial_value + ad_data['Advance_Decline_Diff'].cumsum()
    
    # Calculate percentage of advancing stocks
    ad_data['Pct_Advancing'] = (ad_data['Advances'] / ad_data['Total'] * 100).round(2)
    
    # Calculate moving averages
    ad_data['AD_Line_SMA50'] = ad_data['AD_Line'].rolling(window=50).mean()
    ad_data['AD_Line_SMA200'] = ad_data['AD_Line'].rolling(window=200).mean()
    
    # Calculate Rate of Change (ROC) - 20-day
    ad_data['AD_Line_ROC20'] = ad_data['AD_Line'].pct_change(periods=20) * 100
    
    # Calculate Net Advancing Issues (also known as McClellan Oscillator components)
    ad_data['Net_Advances'] = ad_data['Advances'] - ad_data['Declines']
    ad_data['Net_Advances_EMA19'] = ad_data['Net_Advances'].ewm(span=19).mean()
    ad_data['Net_Advances_EMA39'] = ad_data['Net_Advances'].ewm(span=39).mean()
    
    # Calculate McClellan Oscillator
    ad_data['McClellan_Oscillator'] = ad_data['Net_Advances_EMA19'] - ad_data['Net_Advances_EMA39']
    
    # Calculate Advance/Decline Ratio
    ad_data['AD_Ratio'] = (ad_data['Advances'] / ad_data['Declines']).replace([np.inf, -np.inf], np.nan).fillna(0)
    
    print(f"Calculated A/D line and technical indicators")
    return ad_data

def detect_divergences(ad_data, spy_data, window=60, threshold=0.7):
    """
    Detect divergences between A/D line and SPY.
    
    Args:
        ad_data (pd.DataFrame): A/D line data
        spy_data (pd.DataFrame): SPY price data
        window (int): Window size for divergence detection
        threshold (float): Correlation threshold for divergence
    
    Returns:
        pd.DataFrame: DataFrame with divergence indicators
    """
    # Ensure both datasets have the same index
    ad_data_aligned = ad_data.copy()
    spy_data_aligned = spy_data.copy()
    
    # Get common dates
    common_dates = ad_data_aligned.index.intersection(spy_data_aligned.index)
    
    # Filter data to common dates
    ad_data_aligned = ad_data_aligned.loc[common_dates]
    spy_data_aligned = spy_data_aligned.loc[common_dates]
    
    # Create merged dataframe
    merged_data = pd.DataFrame(index=common_dates)
    merged_data['AD_Line'] = ad_data_aligned['AD_Line']
    merged_data['SPY_Close'] = spy_data_aligned['Close']
    
    # Calculate percentage changes for each
    merged_data['AD_Line_Pct_Change'] = merged_data['AD_Line'].pct_change()
    merged_data['SPY_Pct_Change'] = merged_data['SPY_Close'].pct_change()
    
    # Remove NaN values
    merged_data = merged_data.dropna()
    
    # Rolling correlation between A/D line and SPY
    merged_data['Rolling_Corr'] = merged_data['AD_Line_Pct_Change'].rolling(window=window).corr(
        merged_data['SPY_Pct_Change'])
    
    # Identify potential divergences when correlation falls below threshold
    merged_data['Potential_Divergence'] = (
        merged_data['Rolling_Corr'].abs() < threshold).astype(int)
    
    # Check for periods where A/D line and SPY are moving in opposite directions
    merged_data['Opposite_Direction'] = (
        merged_data['AD_Line_Pct_Change'] * merged_data['SPY_Pct_Change'] < 0).astype(int)
    
    # Combine both conditions for stronger divergence signal
    merged_data['Divergence'] = (
        (merged_data['Potential_Divergence'] == 1) & 
        (merged_data['Opposite_Direction'].rolling(window=5).sum() >= 3)
    ).astype(int)
    
    # Advanced divergence detection
    # Calculate 20-day ROC for both series
    merged_data['AD_Line_ROC20'] = merged_data['AD_Line'].pct_change(periods=20) * 100
    merged_data['SPY_ROC20'] = merged_data['SPY_Close'].pct_change(periods=20) * 100
    
    # Identify classic bearish divergence (price making higher highs, A/D line making lower highs)
    merged_data['Bearish_Divergence'] = ((merged_data['SPY_ROC20'] > 0) & 
                                         (merged_data['AD_Line_ROC20'] < 0)).astype(int)
    
    # Identify classic bullish divergence (price making lower lows, A/D line making higher lows)
    merged_data['Bullish_Divergence'] = ((merged_data['SPY_ROC20'] < 0) & 
                                         (merged_data['AD_Line_ROC20'] > 0)).astype(int)
    
    print(f"Detected divergences between A/D line and SPY")
    return merged_data

def identify_crossovers(ad_data):
    """
    Identifies significant crossovers in the A/D line data.
    
    Args:
        ad_data (pd.DataFrame): DataFrame with A/D line data and SMAs
        
    Returns:
        pd.DataFrame: DataFrame with crossover signals added
    """
    # Create a copy of the input data
    crossover_data = ad_data.copy()
    
    # SMA crossovers (50 and 200-day)
    crossover_data['SMA50_Above_SMA200'] = (crossover_data['AD_Line_SMA50'] > 
                                           crossover_data['AD_Line_SMA200']).astype(int)
    
    # Detect changes in the relationship (crossovers)
    crossover_data['Golden_Cross'] = ((crossover_data['SMA50_Above_SMA200'] == 1) & 
                                    (crossover_data['SMA50_Above_SMA200'].shift(1) == 0)).astype(int)
    
    crossover_data['Death_Cross'] = ((crossover_data['SMA50_Above_SMA200'] == 0) & 
                                   (crossover_data['SMA50_Above_SMA200'].shift(1) == 1)).astype(int)
    
    # Price and SMA crossovers
    crossover_data['AD_Above_SMA50'] = (crossover_data['AD_Line'] > 
                                       crossover_data['AD_Line_SMA50']).astype(int)
    
    crossover_data['AD_Cross_Above_SMA50'] = ((crossover_data['AD_Above_SMA50'] == 1) & 
                                            (crossover_data['AD_Above_SMA50'].shift(1) == 0)).astype(int)
    
    crossover_data['AD_Cross_Below_SMA50'] = ((crossover_data['AD_Above_SMA50'] == 0) & 
                                            (crossover_data['AD_Above_SMA50'].shift(1) == 1)).astype(int)
    
    print("Identified significant A/D line crossover events")
    return crossover_data

def run_ad_line_analysis(start_date=None, end_date=None):
    """
    Main function to run the A/D line analysis from data retrieval to calculations.
    
    Args:
        start_date (str, optional): Start date in 'YYYY-MM-DD' format. Default is 1 year ago.
        end_date (str, optional): End date in 'YYYY-MM-DD' format. Default is today.
        
    Returns:
        tuple: Tuple containing all calculated dataframes
    """
    global nyse_tickers, daily_data, spy_data, breadth_data, ad_data, merged_data, crossover_data
    
    # Set date range for analysis
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    if start_date is None:
        #start_date = (datetime.today() - timedelta(days=365*3)).strftime('%Y-%m-%d')
        start_date='2019-01-01'
    
    print(f"Running A/D line analysis from {start_date} to {end_date}...")
    
    # Get NYSE tickers from CSV file
    nyse_tickers = get_nyse_tickers('NYSE.csv')
    
    # Get daily price data for NYSE stocks
    daily_data = get_daily_data(nyse_tickers, start_date, end_date)
    
    # Get SPY data for comparison
    spy_data = get_spy_data(start_date, end_date)
    
    # Calculate daily advances and declines
    breadth_data = calculate_daily_advances_declines(daily_data)
    
    # Calculate A/D line and technical indicators
    ad_data = calculate_ad_line(breadth_data)
    
    # Detect divergences
    merged_data = detect_divergences(ad_data, spy_data)
    
    # Identify crossovers
    crossover_data = identify_crossovers(ad_data)
    
    # Update ad_data with crossover information
    ad_data = ad_data.join(crossover_data[['Golden_Cross', 'Death_Cross', 'AD_Cross_Above_SMA50', 
                                         'AD_Cross_Below_SMA50']], how='left')
    
    print("A/D line analysis complete!")
    return nyse_tickers, daily_data, spy_data, breadth_data, ad_data, merged_data

# Execute the analysis when running this cell
# Default to analyzing the last year of data
nyse_tickers, daily_data, spy_data, breadth_data, ad_data, merged_data = run_ad_line_analysis()

# Display summary of loaded data
print("\nData Summary:")
print(f"- Total NYSE tickers: {len(nyse_tickers)}")
print(f"- Daily price data shape: {daily_data.shape}")
print(f"- SPY data shape: {spy_data.shape}")
print(f"- Breadth data shape: {breadth_data.shape}")
print(f"- A/D line data shape: {ad_data.shape}")
print(f"- Merged data shape: {merged_data.shape}")

# Display the latest A/D line value
latest_date = ad_data.index[-1]
latest_ad = ad_data.iloc[-1]
print(f"\nLatest A/D Line data (as of {latest_date.strftime('%Y-%m-%d')}):")
print(f"- A/D Line value: {latest_ad['AD_Line']:.2f}")
print(f"- % Advancing: {latest_ad['Pct_Advancing']:.2f}%")
print(f"- 50-day SMA: {latest_ad['AD_Line_SMA50']:.2f}")
print(f"- 200-day SMA: {latest_ad['AD_Line_SMA200']:.2f}")
print(f"- 20-day ROC: {latest_ad['AD_Line_ROC20']:.2f}%")
print(f"- McClellan Oscillator: {latest_ad['McClellan_Oscillator']:.2f}")

Running A/D line analysis from 2019-01-01 to 2025-04-23...
Kept 2356 clean tickers, excluded 380 tickers with special characters
Successfully retrieved 2356 NYSE ticker symbols



3 Failed downloads:
['ETX           ', 'ECC           ']: YFTzMissingError('possibly delisted; no timezone found')
['EAI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-04-23)')



2 Failed downloads:
['SFB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-04-23)')
['SAND          ']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-04-23) (Yahoo error = "No data found, symbol may be delisted")')


Successfully downloaded daily data with shape (1585, 2356)
Successfully downloaded SPY data with columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Calculated daily breadth data with shape (1585, 5)
Calculated A/D line and technical indicators
Detected divergences between A/D line and SPY
Identified significant A/D line crossover events
A/D line analysis complete!

Data Summary:
- Total NYSE tickers: 2356
- Daily price data shape: (1585, 2356)
- SPY data shape: (1585, 5)
- Breadth data shape: (1585, 5)
- A/D line data shape: (1585, 19)
- Merged data shape: (1584, 12)

Latest A/D Line data (as of 2025-04-22):
- A/D Line value: 94769.00
- % Advancing: 90.56%
- 50-day SMA: 96829.18
- 200-day SMA: 96800.57
- 20-day ROC: -3.50%
- McClellan Oscillator: 86.38


/var/folders/0l/68dwwf7n4k14jjgryv9tt5mw0000gn/T/ipykernel_9111/3219284889.py:154: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [11]:
# Cell 2: Visualization Functions
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Set default plotly template for consistent appearance
import plotly.io as pio
pio.templates.default = "plotly_white"

# Define color scheme for consistent visualization
COLORS = {
    'ad_line': '#1F77B4',  # Blue
    'sma50': '#FF7F0E',    # Orange
    'sma200': '#D62728',   # Red
    'spy': '#2CA02C',      # Green
    'bullish': '#00CC96',  # Teal
    'bearish': '#EF553B',  # Red-orange
    'neutral': '#7F7F7F',  # Gray
    'divergence': '#9467BD', # Purple
    'reference': '#BCBD22', # Olive
    'highlight': '#FF9900', # Bright orange
}

def plot_ad_line_main(ad_data, spy_data, merged_data):
    """
    Create main A/D Line visualization with key technical signals.
    
    Args:
        ad_data (pd.DataFrame): A/D line data with technical indicators
        spy_data (pd.DataFrame): SPY price data
        merged_data (pd.DataFrame): Merged data with divergence indicators
    
    Returns:
        plotly.graph_objects.Figure: Interactive A/D Line analysis figure
    """
    # Create subplot with 2 rows
    fig = make_subplots(rows=2, cols=1, 
                         shared_xaxes=True,
                         vertical_spacing=0.1,
                         row_heights=[0.7, 0.3],
                         subplot_titles=("A/D Line with Moving Averages", "SPY Price"))
    
    # Plot A/D Line and SMAs
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line'], 
                   name="A/D Line", 
                   line=dict(color=COLORS['ad_line'], width=2),
                   hovertemplate='%{x}<br>A/D Line: %{y:.0f}<extra></extra>'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line_SMA50'], 
                   name="50-day SMA", 
                   line=dict(color=COLORS['sma50'], width=1.5, dash='dash'),
                   hovertemplate='%{x}<br>50-day SMA: %{y:.0f}<extra></extra>'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line_SMA200'], 
                   name="200-day SMA", 
                   line=dict(color=COLORS['sma200'], width=1.5, dash='dash'),
                   hovertemplate='%{x}<br>200-day SMA: %{y:.0f}<extra></extra>'),
        row=1, col=1
    )
    
    # Ensure SPY data has the same date range as AD line data
    common_dates = ad_data.index.intersection(spy_data.index)
    spy_aligned = spy_data.loc[common_dates]
    
    # Plot SPY price
    fig.add_trace(
        go.Scatter(x=spy_aligned.index, y=spy_aligned['Close'], 
                   name="SPY", 
                   line=dict(color=COLORS['spy'], width=2),
                   hovertemplate='%{x}<br>SPY: $%{y:.2f}<extra></extra>'),
        row=2, col=1
    )
    
    # Highlight Golden Crosses (50-day SMA crosses above 200-day SMA)
    if 'Golden_Cross' in ad_data.columns:
        golden_cross_dates = ad_data[ad_data['Golden_Cross'] == 1].index
        
        for date in golden_cross_dates:
            if date in ad_data.index:
                # Add markers for golden crosses
                fig.add_trace(
                    go.Scatter(x=[date], 
                               y=[ad_data.loc[date, 'AD_Line_SMA50']],
                               mode='markers',
                               marker=dict(symbol='star', size=12, color=COLORS['bullish']),
                               name='Golden Cross',
                               hoverinfo='text',
                               hovertext=f"Golden Cross: {date.strftime('%Y-%m-%d')}",
                               showlegend=False),
                    row=1, col=1
                )
                
                # Add annotations
                fig.add_annotation(
                    x=date,
                    y=ad_data.loc[date, 'AD_Line_SMA50'] * 1.03,
                    text="Golden Cross",
                    showarrow=False,
                    font=dict(color=COLORS['bullish'], size=12),
                    row=1, col=1
                )
    
    # Highlight Death Crosses (50-day SMA crosses below 200-day SMA)
    if 'Death_Cross' in ad_data.columns:
        death_cross_dates = ad_data[ad_data['Death_Cross'] == 1].index
        
        for date in death_cross_dates:
            if date in ad_data.index:
                # Add markers for death crosses
                fig.add_trace(
                    go.Scatter(x=[date], 
                               y=[ad_data.loc[date, 'AD_Line_SMA50']],
                               mode='markers',
                               marker=dict(symbol='x', size=12, color=COLORS['bearish']),
                               name='Death Cross',
                               hoverinfo='text',
                               hovertext=f"Death Cross: {date.strftime('%Y-%m-%d')}",
                               showlegend=False),
                    row=1, col=1
                )
                
                # Add annotations
                fig.add_annotation(
                    x=date,
                    y=ad_data.loc[date, 'AD_Line_SMA50'] * 0.97,
                    text="Death Cross",
                    showarrow=False,
                    font=dict(color=COLORS['bearish'], size=12),
                    row=1, col=1
                )
    
    # Highlight significant divergences (only the strongest ones to avoid clutter)
    if 'Bearish_Divergence' in merged_data.columns:
        # Get only the most significant bearish divergences
        # For this example, we'll take days where the divergence is true and SPY is relatively high
        significant_bearish = merged_data[
            (merged_data['Bearish_Divergence'] == 1) & 
            (merged_data['SPY_Close'] > merged_data['SPY_Close'].shift(10) * 1.02)
        ].index
        
        # Limit to at most 5 significant divergences
        significant_bearish = significant_bearish[-5:] if len(significant_bearish) > 5 else significant_bearish
        
        for date in significant_bearish:
            if date in ad_data.index:
                # Add vertical lines for significant bearish divergences
                fig.add_shape(
                    type="line",
                    x0=date, y0=0, x1=date, 
                    y1=ad_data.loc[date, 'AD_Line'],
                    line=dict(color=COLORS['bearish'], width=1.5, dash="dash"),
                    opacity=0.7,
                    row=1, col=1
                )
                
                # Add annotations
                fig.add_annotation(
                    x=date,
                    y=ad_data.loc[date, 'AD_Line'] * 1.05,
                    text="Bearish Div.",
                    showarrow=False,
                    font=dict(color=COLORS['bearish'], size=11),
                    row=1, col=1
                )
    
    if 'Bullish_Divergence' in merged_data.columns:
        # Get only the most significant bullish divergences
        # For this example, we'll take days where the divergence is true and SPY is relatively low
        significant_bullish = merged_data[
            (merged_data['Bullish_Divergence'] == 1) & 
            (merged_data['SPY_Close'] < merged_data['SPY_Close'].shift(10) * 0.98)
        ].index
        
        # Limit to at most 5 significant divergences
        significant_bullish = significant_bullish[-5:] if len(significant_bullish) > 5 else significant_bullish
        
        for date in significant_bullish:
            if date in ad_data.index:
                # Add vertical lines for significant bullish divergences
                fig.add_shape(
                    type="line",
                    x0=date, y0=0, x1=date, 
                    y1=ad_data.loc[date, 'AD_Line'],
                    line=dict(color=COLORS['bullish'], width=1.5, dash="dash"),
                    opacity=0.7,
                    row=1, col=1
                )
                
                # Add annotations
                fig.add_annotation(
                    x=date,
                    y=ad_data.loc[date, 'AD_Line'] * 0.95,
                    text="Bullish Div.",
                    showarrow=False,
                    font=dict(color=COLORS['bullish'], size=11),
                    row=1, col=1
                )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "S&P 500 Advance/Decline Line Analysis",
            'font': {'size': 24}
        },
        height=800,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5
        ),
        hovermode="x unified",
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="A/D Line Value", row=1, col=1)
    fig.update_yaxes(title_text="SPY Price ($)", row=2, col=1)
    
    # Add descriptive annotation
    fig.add_annotation(
        text="A/D Line tracks market breadth by measuring the cumulative difference between advancing and declining stocks",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def plot_market_regime(ad_data, spy_data):
    """
    Create market regime classification visualization based on A/D Line vs SPY performance.
    
    Args:
        ad_data (pd.DataFrame): A/D line data with technical indicators
        spy_data (pd.DataFrame): SPY price data
    
    Returns:
        plotly.graph_objects.Figure: Interactive market regime figure
    """
    # Ensure data is aligned
    common_dates = ad_data.index.intersection(spy_data.index)
    
    # Filter data to common dates
    ad_aligned = ad_data.loc[common_dates]
    spy_aligned = spy_data.loc[common_dates]
    
    # Create DataFrame for regime analysis
    regime_data = pd.DataFrame(index=common_dates)
    
    # Normalize both to 100 at the start
    first_valid_idx = min(common_dates)
    regime_data['AD_Norm'] = ad_aligned['AD_Line'] / ad_aligned.loc[first_valid_idx, 'AD_Line'] * 100
    regime_data['SPY_Norm'] = spy_aligned['Close'] / spy_aligned.loc[first_valid_idx, 'Close'] * 100
    
    # Relative strength: A/D Line vs SPY
    regime_data['AD_vs_SPY'] = regime_data['AD_Norm'] - regime_data['SPY_Norm']
    
    # 20-day smooth of the relative strength
    regime_data['AD_vs_SPY_Smooth'] = regime_data['AD_vs_SPY'].rolling(window=20).mean()
    
    # Determine regimes based on relative strength
    regime_data['Regime'] = 'Neutral'
    regime_data.loc[regime_data['AD_vs_SPY_Smooth'] > 3, 'Regime'] = 'Strong Breadth'
    regime_data.loc[regime_data['AD_vs_SPY_Smooth'] < -3, 'Regime'] = 'Weak Breadth'
    
    # Change points in regime
    regime_data['Regime_Change'] = (regime_data['Regime'] != regime_data['Regime'].shift(1)).astype(int)
    
    # Create figure
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.1,
                        row_heights=[0.7, 0.3],
                        subplot_titles=("A/D Line vs SPY (Normalized to 100)", 
                                       "Relative Strength (A/D Line - SPY)"))
    
    # Plot normalized A/D Line
    fig.add_trace(
        go.Scatter(x=regime_data.index, y=regime_data['AD_Norm'], 
                  name="A/D Line", 
                  line=dict(color=COLORS['ad_line'], width=2),
                  hovertemplate="%{x}<br>A/D Line: %{y:.1f}<extra></extra>"),
        row=1, col=1
    )
    
    # Plot normalized SPY
    fig.add_trace(
        go.Scatter(x=regime_data.index, y=regime_data['SPY_Norm'], 
                  name="SPY", 
                  line=dict(color=COLORS['spy'], width=2),
                  hovertemplate="%{x}<br>SPY: %{y:.1f}<extra></extra>"),
        row=1, col=1
    )
    
    # Plot regime background colors
    for regime in ['Strong Breadth', 'Weak Breadth', 'Neutral']:
        # Get contiguous periods of the same regime
        mask = regime_data['Regime'] == regime
        
        if mask.any():
            # Convert boolean mask to integer and find transitions
            mask_int = mask.astype(int)
            transitions = np.where(np.diff(np.hstack([[0], mask_int, [0]])))[0]
            
            # Group transitions into (start, end) pairs
            period_pairs = [(transitions[i], transitions[i+1]) for i in range(0, len(transitions), 2)]
            
            # Set colors based on regime
            if regime == 'Strong Breadth':
                color = COLORS['bullish']
                label = "Strong Breadth"
            elif regime == 'Weak Breadth':
                color = COLORS['bearish']
                label = "Weak Breadth"
            else:
                color = COLORS['neutral']
                label = "Neutral"
            
            # Add shaded regions for each period
            for start_idx, end_idx in period_pairs:
                if start_idx < len(regime_data) and end_idx <= len(regime_data):
                    start_date = regime_data.index[start_idx]
                    # Avoid index error by checking bounds
                    end_date = regime_data.index[end_idx-1] if end_idx > 0 and end_idx <= len(regime_data) else regime_data.index[-1]
                    
                    fig.add_shape(
                        type="rect",
                        x0=start_date,
                        x1=end_date,
                        y0=0,
                        y1=200,  # Set higher than max value to ensure coverage
                        fillcolor=color,
                        opacity=0.1,
                        layer="below",
                        line_width=0,
                        row=1, col=1
                    )
    
    # Plot relative strength
    fig.add_trace(
        go.Scatter(x=regime_data.index, y=regime_data['AD_vs_SPY'], 
                  name="A/D - SPY Spread", 
                  line=dict(color=COLORS['divergence'], width=1.5),
                  hovertemplate="%{x}<br>Spread: %{y:.1f}<extra></extra>"),
        row=2, col=1
    )
    
    # Plot smoothed relative strength
    fig.add_trace(
        go.Scatter(x=regime_data.index, y=regime_data['AD_vs_SPY_Smooth'], 
                  name="20-day MA", 
                  line=dict(color=COLORS['divergence'], width=2, dash='dash'),
                  hovertemplate="%{x}<br>20-day MA: %{y:.1f}<extra></extra>"),
        row=2, col=1
    )
    
    # Add horizontal reference lines for regime thresholds
    fig.add_hline(y=3, line_dash="dash", line_color=COLORS['bullish'], row=2, col=1,
                 annotation_text="Strong Breadth Threshold", annotation_position="bottom right")
    fig.add_hline(y=-3, line_dash="dash", line_color=COLORS['bearish'], row=2, col=1,
                 annotation_text="Weak Breadth Threshold", annotation_position="bottom right")
    fig.add_hline(y=0, line_dash="dash", line_color=COLORS['neutral'], row=2, col=1)
    
    # Add 100 reference line in top panel
    fig.add_hline(y=100, line_dash="dash", line_color='gray', row=1, col=1,
                 annotation_text="Starting Value (100)", annotation_position="bottom right")
    
    # Mark regime change points
    regime_changes = regime_data[regime_data['Regime_Change'] == 1]
    for date, row in regime_changes.iterrows():
        fig.add_shape(
            type="line",
            x0=date, y0=0, x1=date, y1=200,
            line=dict(color="black", width=1, dash="dot"),
            opacity=0.7,
            row=1, col=1
        )
        fig.add_shape(
            type="line",
            x0=date, y0=-20, x1=date, y1=20,
            line=dict(color="black", width=1, dash="dot"),
            opacity=0.7,
            row=2, col=1
        )
    
    # Determine current regime
    current_regime = regime_data['Regime'].iloc[-1]
    regime_color = COLORS['neutral']
    if current_regime == 'Strong Breadth':
        regime_color = COLORS['bullish']
    elif current_regime == 'Weak Breadth':
        regime_color = COLORS['bearish']
    
    # Add current regime annotation
    fig.add_annotation(
        text=f"Current Regime: {current_regime}",
        xref="paper", yref="paper",
        x=1.0, y=1.1,
        showarrow=False,
        font=dict(size=16, color=regime_color),
        align="right",
        bordercolor=regime_color,
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Market Regime Classification: A/D Line vs SPY Performance",
            'font': {'size': 24}
        },
        height=800,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5
        ),
        hovermode="x unified",
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Normalized Value (100 = Start)", row=1, col=1)
    fig.update_yaxes(title_text="Relative Performance (A/D - SPY)", row=2, col=1)
    
    # Add descriptive annotation
    fig.add_annotation(
        text="Regime classification based on relative performance of A/D Line vs SPY. Strong breadth regimes typically favor broad-based rallies.",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def plot_breadth_participation(ad_data):
    """
    Create visualization for market breadth participation.
    
    Args:
        ad_data (pd.DataFrame): A/D line data with percentage of advancing stocks
    
    Returns:
        plotly.graph_objects.Figure: Interactive breadth participation figure
    """
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Plot daily percentage of advancing stocks
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['Pct_Advancing'], 
                   name="% Advancing", 
                   line=dict(color=COLORS['ad_line'], width=1),
                   opacity=0.7,
                   hovertemplate="%{x}<br>% Advancing: %{y:.1f}%<extra></extra>"),
        secondary_y=False
    )
    
    # Add 10-day moving average
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['Pct_Advancing'].rolling(10).mean(), 
                   name="10-day MA", 
                   line=dict(color=COLORS['ad_line'], width=2.5),
                   hovertemplate="%{x}<br>10-day MA: %{y:.1f}%<extra></extra>"),
        secondary_y=False
    )
    
    # Calculate McClellan Oscillator and add to chart
    if 'McClellan_Oscillator' in ad_data.columns:
        fig.add_trace(
            go.Scatter(x=ad_data.index, y=ad_data['McClellan_Oscillator'], 
                       name="McClellan Oscillator", 
                       line=dict(color=COLORS['divergence'], width=1.5),
                       hovertemplate="%{x}<br>McClellan: %{y:.1f}<extra></extra>"),
            secondary_y=True
        )
    
    # Add 50% reference line
    fig.add_hline(y=50, line_dash="dash", line_color=COLORS['neutral'],
                 annotation_text="50% Advancing", annotation_position="bottom right")
    
    # Add horizontal reference lines for extreme readings
    # Determine 10th and 90th percentile of Pct_Advancing
    p10 = ad_data['Pct_Advancing'].quantile(0.10)
    p90 = ad_data['Pct_Advancing'].quantile(0.90)
    
    fig.add_hline(y=p10, line_dash="dash", line_color=COLORS['bearish'],
                 annotation_text=f"Extreme Weakness (<{p10:.1f}%)", annotation_position="bottom right")
    fig.add_hline(y=p90, line_dash="dash", line_color=COLORS['bullish'],
                 annotation_text=f"Extreme Strength (>{p90:.1f}%)", annotation_position="bottom right")
    
    # Highlight extreme readings
    # Oversold periods (consecutive days below 10th percentile)
    oversold_mask = ad_data['Pct_Advancing'] < p10
    oversold_starts = ad_data.index[np.where(np.diff(np.hstack([[0], oversold_mask.astype(int), [0]])) == 1)[0]]
    oversold_ends = ad_data.index[np.where(np.diff(np.hstack([[0], oversold_mask.astype(int), [0]])) == -1)[0] - 1]
    
    for start, end in zip(oversold_starts, oversold_ends):
        # Only highlight if period is at least 2 days
        if (end - start).days >= 2:
            fig.add_shape(
                type="rect",
                x0=start, x1=end,
                y0=0, y1=p10,
                fillcolor=COLORS['bearish'],
                opacity=0.3,
                layer="below",
                line_width=0
            )
    
    # Overbought periods (consecutive days above 90th percentile)
    overbought_mask = ad_data['Pct_Advancing'] > p90
    overbought_starts = ad_data.index[np.where(np.diff(np.hstack([[0], overbought_mask.astype(int), [0]])) == 1)[0]]
    overbought_ends = ad_data.index[np.where(np.diff(np.hstack([[0], overbought_mask.astype(int), [0]])) == -1)[0] - 1]
    
    for start, end in zip(overbought_starts, overbought_ends):
        # Only highlight if period is at least 2 days
        if (end - start).days >= 2:
            fig.add_shape(
                type="rect",
                x0=start, x1=end,
                y0=p90, y1=100,
                fillcolor=COLORS['bullish'],
                opacity=0.3,
                layer="below",
                line_width=0
            )
    
    # Add breadth thrusts (days with exceptional advances)
    breadth_thrusts = ad_data[ad_data['Pct_Advancing'] > 80].index
    for date in breadth_thrusts:
        fig.add_trace(
            go.Scatter(x=[date], 
                       y=[ad_data.loc[date, 'Pct_Advancing']],
                       mode='markers',
                       marker=dict(symbol='triangle-up', size=12, color=COLORS['highlight']),
                       name='Breadth Thrust',
                       hoverinfo='text',
                       hovertext=f"Breadth Thrust: {date.strftime('%Y-%m-%d')} - {ad_data.loc[date, 'Pct_Advancing']:.1f}%",
                       showlegend=False)
        )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Market Breadth Participation Analysis",
            'font': {'size': 24}
        },
        height=600,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5
        ),
        hovermode="x unified",
    )
    
    # Set y-axes titles
    fig.update_yaxes(title_text="% of Stocks Advancing", secondary_y=False)
    fig.update_yaxes(title_text="McClellan Oscillator", secondary_y=True)
    
    # Add descriptive annotation
    fig.add_annotation(
        text="Percentage of advancing stocks shows daily market participation. Breadth thrusts (>80% advancing) often signal strong momentum.",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def plot_divergence_analysis(ad_data, spy_data, merged_data):
    """
    Create visualization for divergence and correlation analysis.
    
    Args:
        ad_data (pd.DataFrame): A/D line data with technical indicators
        spy_data (pd.DataFrame): SPY price data
        merged_data (pd.DataFrame): Merged data with divergence indicators
    
    Returns:
        plotly.graph_objects.Figure: Interactive divergence analysis figure
    """
    # Create figure with subplots
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.1,
                        row_heights=[0.6, 0.4],
                        subplot_titles=("A/D Line & SPY 20-day Rate of Change", 
                                       "Rolling 60-day Correlation"))
    
    # Ensure merged data has the necessary columns
    if 'AD_Line_ROC20' not in merged_data.columns or 'SPY_ROC20' not in merged_data.columns:
        # Calculate 20-day ROC for both if needed
        if 'AD_Line' in merged_data.columns and 'SPY_Close' in merged_data.columns:
            merged_data['AD_Line_ROC20'] = merged_data['AD_Line'].pct_change(periods=20) * 100
            merged_data['SPY_ROC20'] = merged_data['SPY_Close'].pct_change(periods=20) * 100
    
    # Plot A/D Line 20-day ROC
    fig.add_trace(
        go.Scatter(x=merged_data.index, y=merged_data['AD_Line_ROC20'], 
                   name="A/D Line 20-day ROC", 
                   line=dict(color=COLORS['ad_line'], width=2),
                   hovertemplate="%{x}<br>A/D Line ROC: %{y:.2f}%<extra></extra>"),
        row=1, col=1
    )
    
    # Plot SPY 20-day ROC
    fig.add_trace(
        go.Scatter(x=merged_data.index, y=merged_data['SPY_ROC20'], 
                   name="SPY 20-day ROC", 
                   line=dict(color=COLORS['spy'], width=2),
                   hovertemplate="%{x}<br>SPY ROC: %{y:.2f}%<extra></extra>"),
        row=1, col=1
    )
    
    # Add zero line
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)
    
    # Plot rolling correlation
    if 'Rolling_Corr' in merged_data.columns:
        fig.add_trace(
            go.Scatter(x=merged_data.index, y=merged_data['Rolling_Corr'], 
                       name="60-day Rolling Correlation", 
                       line=dict(color=COLORS['divergence'], width=2),
                       hovertemplate="%{x}<br>Correlation: %{y:.2f}<extra></extra>"),
            row=2, col=1
        )
        
        # Add correlation reference lines
        fig.add_hline(y=0.7, line_dash="dash", line_color=COLORS['bullish'], row=2, col=1,
                     annotation_text="Strong Positive", annotation_position="bottom right")
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
        fig.add_hline(y=-0.7, line_dash="dash", line_color=COLORS['bearish'], row=2, col=1,
                     annotation_text="Strong Negative", annotation_position="bottom right")
    
    # Highlight periods of significant divergence
    if 'Bearish_Divergence' in merged_data.columns and 'Bullish_Divergence' in merged_data.columns:
        # Identify consecutive days of bearish divergence
        bearish_mask = merged_data['Bearish_Divergence'] == 1
        bearish_starts = merged_data.index[np.where(np.diff(np.hstack([[0], bearish_mask.astype(int), [0]])) == 1)[0]]
        bearish_ends = merged_data.index[np.where(np.diff(np.hstack([[0], bearish_mask.astype(int), [0]])) == -1)[0] - 1]
        
        for start, end in zip(bearish_starts, bearish_ends):
            # Only highlight if period is at least 3 days
            if (end - start).days >= 3:
                fig.add_shape(
                    type="rect",
                    x0=start, x1=end,
                    y0=-25, y1=25,  # Adjusted to cover typical ROC range
                    fillcolor=COLORS['bearish'],
                    opacity=0.2,
                    layer="below",
                    line_width=0,
                    row=1, col=1
                )
                
                # Add annotation for first significant bearish divergence
                if start == bearish_starts[0]:
                    fig.add_annotation(
                        x=start + (end - start)/2,
                        y=20,  # Position above the data
                        text="Bearish Divergence",
                        showarrow=False,
                        font=dict(color=COLORS['bearish'], size=12),
                        bgcolor="white",
                        opacity=0.8,
                        row=1, col=1
                    )
        
        # Identify consecutive days of bullish divergence
        bullish_mask = merged_data['Bullish_Divergence'] == 1
        bullish_starts = merged_data.index[np.where(np.diff(np.hstack([[0], bullish_mask.astype(int), [0]])) == 1)[0]]
        bullish_ends = merged_data.index[np.where(np.diff(np.hstack([[0], bullish_mask.astype(int), [0]])) == -1)[0] - 1]
        
        for start, end in zip(bullish_starts, bullish_ends):
            # Only highlight if period is at least 3 days
            if (end - start).days >= 3:
                fig.add_shape(
                    type="rect",
                    x0=start, x1=end,
                    y0=-25, y1=25,  # Adjusted to cover typical ROC range
                    fillcolor=COLORS['bullish'],
                    opacity=0.2,
                    layer="below",
                    line_width=0,
                    row=1, col=1
                )
                
                # Add annotation for first significant bullish divergence
                if start == bullish_starts[0]:
                    fig.add_annotation(
                        x=start + (end - start)/2,
                        y=-20,  # Position below the data
                        text="Bullish Divergence",
                        showarrow=False,
                        font=dict(color=COLORS['bullish'], size=12),
                        bgcolor="white",
                        opacity=0.8,
                        row=1, col=1
                    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Divergence and Correlation Analysis",
            'font': {'size': 24}
        },
        height=800,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5
        ),
        hovermode="x unified",
    )
    
    # Set y-axes titles
    fig.update_yaxes(title_text="20-day Rate of Change (%)", row=1, col=1)
    fig.update_yaxes(title_text="60-day Rolling Correlation", row=2, col=1)
    
    # Add descriptive annotation
    fig.add_annotation(
        text="Divergences occur when price and breadth move in opposite directions, often signaling potential trend reversals.",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def display_breadth_summary(ad_data, spy_data, merged_data):
    """
    Create and display a summary of current market breadth conditions.
    
    Args:
        ad_data (pd.DataFrame): A/D line data with technical indicators
        spy_data (pd.DataFrame): SPY price data
        merged_data (pd.DataFrame): Merged data with divergence indicators
    
    Returns:
        None: Displays summary directly using IPython.display
    """
    # Get latest data
    latest_date = ad_data.index[-1]
    latest_ad = ad_data.iloc[-1]
    
    # Create breadth health score (0-100) based on multiple factors
    health_score = 50  # Start at neutral
    
    # Factor 1: A/D Line vs. 50-day SMA
    if latest_ad['AD_Line'] > latest_ad['AD_Line_SMA50']:
        health_score += 15
    else:
        health_score -= 15
    
    # Factor 2: A/D Line vs. 200-day SMA
    if latest_ad['AD_Line'] > latest_ad['AD_Line_SMA200']:
        health_score += 10
    else:
        health_score -= 10
    
    # Factor 3: 50-day SMA vs. 200-day SMA
    if latest_ad['AD_Line_SMA50'] > latest_ad['AD_Line_SMA200']:
        health_score += 10
    else:
        health_score -= 10
    
    # Factor 4: 20-day ROC
    if latest_ad['AD_Line_ROC20'] > 0:
        health_score += 5
        if latest_ad['AD_Line_ROC20'] > 5:
            health_score += 5
    else:
        health_score -= 5
        if latest_ad['AD_Line_ROC20'] < -5:
            health_score -= 5
    
    # Factor 5: McClellan Oscillator
    if 'McClellan_Oscillator' in latest_ad:
        if latest_ad['McClellan_Oscillator'] > 0:
            health_score += 5
            if latest_ad['McClellan_Oscillator'] > 50:
                health_score += 5
        else:
            health_score -= 5
            if latest_ad['McClellan_Oscillator'] < -50:
                health_score -= 5
    
    # Ensure health score stays within 0-100 range
    health_score = max(0, min(100, health_score))
    
    # Determine market breadth condition based on health score
    if health_score >= 80:
        breadth_condition = "Very Bullish"
        condition_color = COLORS['bullish']
    elif health_score >= 60:
        breadth_condition = "Bullish"
        condition_color = COLORS['bullish']
    elif health_score >= 40:
        breadth_condition = "Neutral"
        condition_color = COLORS['neutral']
    elif health_score >= 20:
        breadth_condition = "Bearish"
        condition_color = COLORS['bearish']
    else:
        breadth_condition = "Very Bearish"
        condition_color = COLORS['bearish']
    
    # Generate text insights
    insights = []
    
    # Insight 1: Current A/D Line trend
    if latest_ad['AD_Line'] > latest_ad['AD_Line_SMA50'] and latest_ad['AD_Line_SMA50'] > latest_ad['AD_Line_SMA200']:
        insights.append("A/D Line is in a strong uptrend, above both 50-day and 200-day SMAs.")
    elif latest_ad['AD_Line'] > latest_ad['AD_Line_SMA50']:
        insights.append("A/D Line is above its 50-day SMA, showing short-term strength.")
    elif latest_ad['AD_Line'] < latest_ad['AD_Line_SMA50'] and latest_ad['AD_Line_SMA50'] < latest_ad['AD_Line_SMA200']:
        insights.append("A/D Line is in a strong downtrend, below both 50-day and 200-day SMAs.")
    elif latest_ad['AD_Line'] < latest_ad['AD_Line_SMA50']:
        insights.append("A/D Line is below its 50-day SMA, showing short-term weakness.")
    
    # Insight 2: Recent crossovers
    last_n_days = min(60, len(ad_data))
    recent_data = ad_data.iloc[-last_n_days:]
    
    if 'Golden_Cross' in recent_data.columns and recent_data['Golden_Cross'].sum() > 0:
        golden_cross_date = recent_data[recent_data['Golden_Cross'] == 1].index[0]
        insights.append(f"Golden Cross occurred on {golden_cross_date.strftime('%Y-%m-%d')}, a traditionally bullish signal.")
    elif 'Death_Cross' in recent_data.columns and recent_data['Death_Cross'].sum() > 0:
        death_cross_date = recent_data[recent_data['Death_Cross'] == 1].index[0]
        insights.append(f"Death Cross occurred on {death_cross_date.strftime('%Y-%m-%d')}, a traditionally bearish signal.")
    
    # Insight 3: Recent divergences
    if 'Bearish_Divergence' in merged_data.columns and merged_data['Bearish_Divergence'].iloc[-30:].sum() > 5:
        insights.append("Multiple bearish divergences detected in the past month, suggesting potential weakness ahead.")
    elif 'Bullish_Divergence' in merged_data.columns and merged_data['Bullish_Divergence'].iloc[-30:].sum() > 5:
        insights.append("Multiple bullish divergences detected in the past month, suggesting potential strength ahead.")
    
    # Insight 4: Market participation
    recent_pct_advancing = ad_data['Pct_Advancing'].iloc[-10:].mean()
    if recent_pct_advancing > 60:
        insights.append(f"Strong market participation with {recent_pct_advancing:.1f}% of stocks advancing on average over the past 10 days.")
    elif recent_pct_advancing < 40:
        insights.append(f"Weak market participation with only {recent_pct_advancing:.1f}% of stocks advancing on average over the past 10 days.")
    
    # Insight 5: McClellan Oscillator
    if 'McClellan_Oscillator' in latest_ad:
        recent_mcclellan = latest_ad['McClellan_Oscillator']
        if recent_mcclellan > 100:
            insights.append(f"McClellan Oscillator at {recent_mcclellan:.0f}, showing very strong breadth momentum.")
        elif recent_mcclellan > 50:
            insights.append(f"McClellan Oscillator at {recent_mcclellan:.0f}, showing positive breadth momentum.")
        elif recent_mcclellan < -100:
            insights.append(f"McClellan Oscillator at {recent_mcclellan:.0f}, showing very weak breadth momentum.")
        elif recent_mcclellan < -50:
            insights.append(f"McClellan Oscillator at {recent_mcclellan:.0f}, showing negative breadth momentum.")
    
    # Display summary using Markdown
    html_content = f"""
    <div style="font-family: Arial; padding: 20px; border-radius: 10px; border: 1px solid #ddd; background-color: #f9f9f9;">
        <h1 style="text-align: center; margin-bottom: 20px;">Market Breadth Summary</h1>
        
        <div style="display: flex; justify-content: space-between; margin-bottom: 30px;">
            <div style="flex: 1; padding: 10px;">
                <h3>Latest Data (as of {latest_date.strftime('%Y-%m-%d')})</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td>A/D Line Value:</td><td style="text-align: right;"><b>{latest_ad['AD_Line']:.2f}</b></td></tr>
                    <tr><td>% Advancing:</td><td style="text-align: right;"><b>{latest_ad['Pct_Advancing']:.2f}%</b></td></tr>
                    <tr><td>50-day SMA:</td><td style="text-align: right;"><b>{latest_ad['AD_Line_SMA50']:.2f}</b></td></tr>
                    <tr><td>200-day SMA:</td><td style="text-align: right;"><b>{latest_ad['AD_Line_SMA200']:.2f}</b></td></tr>
                    <tr><td>20-day ROC:</td><td style="text-align: right;"><b>{latest_ad['AD_Line_ROC20']:.2f}%</b></td></tr>
                </table>
            </div>
            
            <div style="flex: 1; padding: 10px; text-align: center;">
                <h3>Market Breadth Health</h3>
                <div style="margin: 20px auto; width: 200px; height: 200px; border-radius: 50%; background: conic-gradient({condition_color} 0% {health_score}%, #f1f1f1 {health_score}% 100%); position: relative;">
                    <div style="position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); font-size: 48px; font-weight: bold; color: {condition_color};">{health_score}</div>
                </div>
                <div style="font-size: 24px; font-weight: bold; color: {condition_color};">{breadth_condition}</div>
            </div>
        </div>
        
        <div style="margin-top: 20px;">
            <h3>Market Breadth Insights</h3>
            <ul>
    """
    
    for insight in insights:
        html_content += f"<li>{insight}</li>"
    
    html_content += """
            </ul>
        </div>
    </div>
    """
    
    display(HTML(html_content))

# Execute the visualizations
print("Generating visualizations...")

# Display section header
display(Markdown("# S&P 500 Advance/Decline Line Analysis"))
display(Markdown("## 1. A/D Line Primary Analysis"))

# Plot main A/D Line analysis
fig1 = plot_ad_line_main(ad_data, spy_data, merged_data)
fig1.show()

# Display section header
display(Markdown("## 2. Market Regime Classification"))

# Plot market regime analysis
fig2 = plot_market_regime(ad_data, spy_data)
fig2.show()

# Display section header
display(Markdown("## 3. Breadth Participation Analysis"))

# Plot breadth participation
fig3 = plot_breadth_participation(ad_data)
fig3.show()

# Display section header
display(Markdown("## 4. Divergence and Correlation Analysis"))

# Plot divergence analysis
fig4 = plot_divergence_analysis(ad_data, spy_data, merged_data)
fig4.show()

# Display section header
display(Markdown("## 5. Market Breadth Summary"))

# Display breadth summary
display_breadth_summary(ad_data, spy_data, merged_data)

print("Visualization complete!")

Generating visualizations...


# S&P 500 Advance/Decline Line Analysis

## 1. A/D Line Primary Analysis

## 2. Market Regime Classification

## 3. Breadth Participation Analysis

## 4. Divergence and Correlation Analysis

## 5. Market Breadth Summary

A/D Line Value:,94769.00
% Advancing:,90.56%
50-day SMA:,96829.18
200-day SMA:,96800.57
20-day ROC:,-3.50%


Visualization complete!
